In [1]:
import pandas as pd
import thinkplot
import numpy as np
import re
import thinkstats2

Helper functions

`clean_name` is used to clean occupation names by removing special characters and only capitalizing the first letter

In [2]:
def clean_name(x):
    if type(x) != str:
        x = str(x)
    x = re.sub(r'([^\s\w]|_)+', '', x)
    x = x.lower()
    return x.capitalize()

`clean_val` is used to convert a string to an integer after removing ',' and '$'

In [3]:
def clean_val(val):
    if type(val) == str:
        val = val.replace('$', '')
        val = val.replace(',', '')
        val = int(val)
    return val

`get_header_names` returns the header names for an inputted year

In [4]:
def get_header_names(yr):
    new_header_names = []
    header_names = ['Occupation', 'Total Number of workers', 'Total Median weekly earnings', 'Men Number of workers', 'Men Median weekly earnings', 'Women Number of workers', 'Women Median weekly earnings']
    for header in header_names:
        if header == 'Occupation':
            new_header_names.append(header)
        else:
            new_header_names.append(str(yr)+' '+header)
    return new_header_names

Read in txt files and convert to excel files

In [5]:
def clean_txt(yr):
    header_names = get_header_names(yr)
    df_raw = pd.read_fwf('Data/weeklyincome_occupation_gender_'+str(yr)+'.txt', names=header_names)
    # Drop rows with no occupation
    df_raw = df_raw[df_raw['Occupation'].notnull()]
    # Clean occupation names
    df_raw['Occupation'] = df_raw['Occupation'].apply(clean_name)
    # Consolidate rows with really long occ names
    df_occ_names = df_raw[df_raw[str(yr)+' Total Number of workers'].isnull()]
    for ind in df_occ_names.index:
        df_raw.loc[ind+1]['Occupation'] = df_raw.loc[ind]['Occupation']+df_raw.loc[ind+1]['Occupation']
    df_raw = df_raw[df_raw[str(yr)+' Total Number of workers'].notnull()]
    # Replace (1) and - with NaN
    df_raw = df_raw.replace('(1)', np.nan)
    df_raw = df_raw.replace('-', np.nan)
    df_raw = df_raw.dropna()
    # Clean values
    for header in header_names[1:]:
        df_raw[header] = df_raw[header].apply(clean_val)
    #df_raw.to_excel('Data/weeklyincome_occupation_gender_clean_'+str(yr)+'.xlsx')
    return df_raw

Run `clean_txt` from 2002 to 2010

In [6]:
years = range(2002, 2011)
for yr in years:
    clean_txt(yr)

Combine the data from 2003 to 2010 (we removed 2002 since it uses other occupation codes)

In [7]:
years = range(2003, 2011)
dfs = {}
for yr in years:
    df = pd.read_excel('Data/weeklyincome_occupation_gender_clean_'+str(yr)+'.xlsx', index=False)
    df = df.set_index('Occupation')
    dfs[yr] = df

Columns are arranged from 2010 to 2003

In [8]:
df_combined_2002 = dfs[2010]
for yr in reversed(years[:-1]):
    df_combined_2002 = pd.concat([df_combined_2002, dfs[yr]], axis=1, join_axes=[df_combined_2002.index])
df_combined_2002.head()

,2010 Total Number of workers,2010 Total Median weekly earnings,2010 Men Number of workers,2010 Men Median weekly earnings,2010 Women Number of workers,2010 Women Median weekly earnings,2009 Total Number of workers,2009 Total Median weekly earnings,2009 Men Number of workers,2009 Men Median weekly earnings,...,2004 Men Number of workers,2004 Men Median weekly earnings,2004 Women Number of workers,2004 Women Median weekly earnings,2003 Total Number of workers,2003 Total Median weekly earnings,2003 Men Number of workers,2003 Men Median weekly earnings,2003 Women Number of workers,2003 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,99531,747,55059,824,44472,669,99820.0,739.0,55108.0,819.0,...,57001.0,713.0,44223.0,573.0,100302.0,620.0,56227.0,695.0,44076.0,552.0
Management professional and related occupations,39145,1063,19009,1256,20136,923,39080.0,1044.0,18928.0,1248.0,...,17981.0,1098.0,18168.0,780.0,35680.0,887.0,17718.0,1059.0,17962.0,758.0
Management business and financial operations occupations,15648,1155,8552,1363,7096,971,15879.0,1138.0,8635.0,1334.0,...,8170.0,1158.0,6609.0,812.0,14493.0,961.0,8047.0,1143.0,6446.0,799.0
Management occupations,10743,1230,6376,1414,4368,1018,10907.0,1208.0,6508.0,1384.0,...,6226.0,1215.0,3995.0,871.0,10115.0,1023.0,6143.0,1172.0,3973.0,849.0
Chief executives,1034,1949,769,2217,265,1598,1105.0,1916.0,837.0,2084.0,...,802.0,1875.0,248.0,1310.0,1040.0,1558.0,796.0,1736.0,244.0,1243.0


In [9]:
df_combined_2002 = df_combined_2002.dropna()

In [10]:
#df_combined_2002.to_excel('Data/weeklyincome_occupation_gender_combined_2002codes.xlsx')

Find percent female in each occupation and accumulate data in array form

In [11]:
df_combined_2002 = pd.read_excel('Data/weeklyincome_occupation_gender_combined_2002codes.xlsx')
df_combined_2002 = df_combined_2002.set_index('Occupation')
df_combined_2002.head()

,2010 Total Number of workers,2010 Total Median weekly earnings,2010 Men Number of workers,2010 Men Median weekly earnings,2010 Women Number of workers,2010 Women Median weekly earnings,2009 Total Number of workers,2009 Total Median weekly earnings,2009 Men Number of workers,2009 Men Median weekly earnings,...,2004 Men Number of workers,2004 Men Median weekly earnings,2004 Women Number of workers,2004 Women Median weekly earnings,2003 Total Number of workers,2003 Total Median weekly earnings,2003 Men Number of workers,2003 Men Median weekly earnings,2003 Women Number of workers,2003 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,99531,747,55059,824,44472,669,99820,739,55108,819,...,57001,713,44223,573,100302,620,56227,695,44076,552
Management professional and related occupations,39145,1063,19009,1256,20136,923,39080,1044,18928,1248,...,17981,1098,18168,780,35680,887,17718,1059,17962,758
Management business and financial operations occupations,15648,1155,8552,1363,7096,971,15879,1138,8635,1334,...,8170,1158,6609,812,14493,961,8047,1143,6446,799
Management occupations,10743,1230,6376,1414,4368,1018,10907,1208,6508,1384,...,6226,1215,3995,871,10115,1023,6143,1172,3973,849
Chief executives,1034,1949,769,2217,265,1598,1105,1916,837,2084,...,802,1875,248,1310,1040,1558,796,1736,244,1243


For every year, find percent female and pay gap in each occupation. Also accumulate the names of the weekly pay columns to append them together.

In [12]:
def consolidate_df(df, years):
    df_new = pd.DataFrame(index=df.index)
    df_new['Percent Female'] = pd.Series()
    df_new['Pay Gap'] = pd.Series()
    df_new['Total Weekly Pay'] = pd.Series()
    df_new['Female Weekly Pay'] = pd.Series()
    df_new['Male Weekly Pay'] = pd.Series()
    
    # Find percent female for each year and create arrays of column names
    percent_female_names = []
    total_weekly_pay_names = []
    female_weekly_pay_names = []
    male_weekly_pay_names = []
    pay_gap_names = []

    for yr in years:
        total_num = df[str(yr)+' Total Number of workers']
        female_num = df[str(yr)+' Women Number of workers']
        df[str(yr)+' Percent Female'] = (female_num/total_num)*100
        
        female_pay = df[str(yr)+' Women Median weekly earnings']
        male_pay = df[str(yr)+' Men Median weekly earnings']
        df[str(yr)+' Pay Gap'] = male_pay - female_pay
    
        percent_female_names.append(str(yr)+' Percent Female')
        total_weekly_pay_names.append(str(yr)+' Total Median weekly earnings')
        female_weekly_pay_names.append(str(yr)+' Women Median weekly earnings')
        male_weekly_pay_names.append(str(yr)+' Men Median weekly earnings')
        pay_gap_names.append(str(yr)+' Pay Gap')
        
    # Append columns
    df_new['Percent Female'] = df[percent_female_names].values.tolist()
    df_new['Total Weekly Pay'] = df[total_weekly_pay_names].values.tolist()
    df_new['Female Weekly Pay'] = df[female_weekly_pay_names].values.tolist()
    df_new['Male Weekly Pay'] = df[male_weekly_pay_names].values.tolist()
    df_new['Pay Gap'] = df[pay_gap_names].values.tolist()
    
    return df_new

In [13]:
df_combined_2002_clean = consolidate_df(df_combined_2002, range(2010, 2002, -1))
df_combined_2002_clean.head()

,Percent Female,Pay Gap,Total Weekly Pay,Female Weekly Pay,Male Weekly Pay
Occupation,,,,,
Total fulltime wage and salary workers,"[44.68155649998493, 44.792626728110605, 44.266...","[155, 162, 160, 152, 143, 137, 140, 143]","[747, 739, 722, 695, 671, 651, 638, 620]","[669, 657, 638, 614, 600, 585, 573, 552]","[824, 819, 798, 766, 743, 722, 713, 695]"
Management professional and related occupations,"[51.43951973432112, 51.566018423746165, 51.140...","[333, 341, 346, 329, 314, 300, 318, 301]","[1063, 1044, 1025, 996, 967, 937, 918, 887]","[923, 907, 892, 858, 840, 813, 780, 758]","[1256, 1248, 1238, 1187, 1154, 1113, 1098, 1059]"
Management business and financial operations occupations,"[45.347648261758685, 45.62000125952516, 45.757...","[392, 379, 402, 353, 350, 320, 346, 344]","[1155, 1138, 1128, 1080, 1045, 997, 965, 961]","[971, 955, 941, 908, 881, 847, 812, 799]","[1363, 1334, 1343, 1261, 1231, 1167, 1158, 1143]"
Management occupations,"[40.65903378944429, 40.331896946914824, 40.408...","[396, 382, 405, 374, 338, 328, 344, 323]","[1230, 1208, 1204, 1161, 1127, 1083, 1052, 1023]","[1018, 1002, 979, 963, 926, 902, 871, 849]","[1414, 1384, 1384, 1337, 1264, 1230, 1215, 1172]"
Chief executives,"[25.62862669245648, 24.25339366515837, 24.0421...","[619, 531, 396, 382, 485, 490, 565, 493]","[1949, 1916, 1903, 1882, 1875, 1834, 1663, 1558]","[1598, 1553, 1603, 1536, 1422, 1413, 1310, 1243]","[2217, 2084, 1999, 1918, 1907, 1903, 1875, 1736]"


In [14]:
#df_combined_2002_clean.to_excel('Data/weeklyincome_occupation_gender_combined_clean_2002codes.xlsx')

Read and clean excel files

In [15]:
def clean_excel(yr):
    df_raw = pd.read_excel('Data/weeklyincome_occupation_gender_'+str(yr)+'.xlsx')
    # drop rows with no occupation
    df_raw = df_raw.dropna()
    # clean occupation names
    df_raw['Occupation'] = df_raw['Occupation'].apply(clean_name)
    # Replace '–' with NaN
    df_raw = df_raw.replace('–', np.nan)
    df_raw = df_raw.replace('-', np.nan)
    df_raw = df_raw.dropna()
    # Write to excel
    #df_raw.to_excel('Data/weeklyincome_occupation_gender_clean_'+str(yr)+'.xlsx')
    return df_raw

Clean excel years 2011 to 2018

In [16]:
years = range(2011, 2019)
for yr in years:
    clean_excel(yr)

Read in data with occupation as index and create dictionary of dfs

In [17]:
years = range(2011, 2019)
dfs = {}
for yr in years:
    df = pd.read_excel('Data/weeklyincome_occupation_gender_clean_'+str(yr)+'.xlsx', index=False)
    df = df.set_index('Occupation')
    dfs[yr] = df

Combine dfs with 2018 being first and 2011 being last

In [18]:
df_combined_2010 = dfs[2018]
for yr in reversed(years[:-1]):
    df_combined_2010 = pd.concat([df_combined_2010, dfs[yr]], axis=1, join_axes=[df_combined_2010.index])
df_combined_2010.head()

,2018 Total Number of workers,2018 Total Median weekly earnings,2018 Men Number of workers,2018 Men Median weekly earnings,2018 Women Number of workers,2018 Women Median weekly earnings,2017 Total Number of workers,2017 Total Median weekly earnings,2017 Men Number of workers,2017 Men Median weekly earnings,...,2012 Men Number of workers,2012 Men Median weekly earnings,2012 Women Number of workers,2012 Women Median weekly earnings,2011 Total Number of workers,2011 Total Median weekly earnings,2011 Men Number of workers,2011 Men Median weekly earnings,2011 Women Number of workers,2011 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,115567,886,64142,973,51425,789,113272.0,860.0,62980.0,941.0,...,57286.0,$854,45462.0,$691,100457.0,$756,55971.0,$832,44486.0,$684
Management professional and related occupations,48808,1246,23685,1468,25123,1078,47207.0,1224.0,22815.0,1442.0,...,19926.0,"1,328",21059.0,951,39791.0,"1,082",19267.0,"1,269",20524.0,941
Management business and financial operations occupations,19863,1355,10668,1537,9195,1168,19414.0,1327.0,10415.0,1526.0,...,9121.0,"1,387",7869.0,993,16061.0,"1,160",8676.0,"1,370",7386.0,977
Management occupations,13477,1429,7754,1585,5724,1236,13169.0,1392.0,7568.0,1573.0,...,6783.0,"1,428",4765.0,"1,036",10891.0,"1,237",6451.0,"1,427",4440.0,"1,018"
Chief executives,1098,2291,790,2488,307,1736,1136.0,2296.0,823.0,2415.0,...,739.0,"2,275",265.0,"1,730",990.0,"1,963",745.0,"2,122",245.0,"1,464"


In [19]:
df_combined_2010 = df_combined_2010.dropna()

In [20]:
#df_combined_2010.to_excel('Data/weeklyincome_occupation_gender_combined_2010codes.xlsx')

Find percent female in each occupation and accumulate data in array form

In [21]:
df_combined_2010 = pd.read_excel('Data/weeklyincome_occupation_gender_combined_2010codes.xlsx')
df_combined_2010 = df_combined_2010.set_index('Occupation')
df_combined_2010.head()

,2018 Total Number of workers,2018 Total Median weekly earnings,2018 Men Number of workers,2018 Men Median weekly earnings,2018 Women Number of workers,2018 Women Median weekly earnings,2017 Total Number of workers,2017 Total Median weekly earnings,2017 Men Number of workers,2017 Men Median weekly earnings,...,2012 Men Number of workers,2012 Men Median weekly earnings,2012 Women Number of workers,2012 Women Median weekly earnings,2011 Total Number of workers,2011 Total Median weekly earnings,2011 Men Number of workers,2011 Men Median weekly earnings,2011 Women Number of workers,2011 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,115567,886,64142,973,51425,789,113272,860,62980,941,...,57286,$854,45462,$691,100457,$756,55971,$832,44486,$684
Management professional and related occupations,48808,1246,23685,1468,25123,1078,47207,1224,22815,1442,...,19926,"1,328",21059,951,39791,"1,082",19267,"1,269",20524,941
Management business and financial operations occupations,19863,1355,10668,1537,9195,1168,19414,1327,10415,1526,...,9121,"1,387",7869,993,16061,"1,160",8676,"1,370",7386,977
Management occupations,13477,1429,7754,1585,5724,1236,13169,1392,7568,1573,...,6783,"1,428",4765,"1,036",10891,"1,237",6451,"1,427",4440,"1,018"
Chief executives,1098,2291,790,2488,307,1736,1136,2296,823,2415,...,739,"2,275",265,"1,730",990,"1,963",745,"2,122",245,"1,464"


In [22]:
for col in df_combined_2010.columns:
    df_combined_2010[col] = df_combined_2010[col].apply(clean_val)

Go through every occupation and find proportion of women workers and median weekly pay (total, female, and male)

In [23]:
df_combined_2010_clean = consolidate_df(df_combined_2010, range(2018, 2010, -1))
df_combined_2010_clean.head()

,Percent Female,Pay Gap,Total Weekly Pay,Female Weekly Pay,Male Weekly Pay
Occupation,,,,,
Total fulltime wage and salary workers,"[44.497996833005956, 44.39843915530758, 44.252...","[184, 171, 166, 169, 152, 154, 163, 148]","[886, 860, 832, 809, 791, 776, 768, 756]","[789, 770, 749, 726, 719, 706, 691, 684]","[973, 941, 915, 895, 871, 860, 854, 832]"
Management professional and related occupations,"[51.47311916079331, 51.672421462918635, 51.543...","[390, 390, 393, 387, 365, 376, 377, 328]","[1246, 1224, 1188, 1158, 1137, 1132, 1108, 1082]","[1078, 1052, 1027, 996, 981, 973, 951, 941]","[1468, 1442, 1420, 1383, 1346, 1349, 1328, 1269]"
Management business and financial operations occupations,"[46.29210089110406, 46.35314721335119, 46.0547...","[369, 392, 392, 363, 360, 363, 394, 393]","[1355, 1327, 1284, 1258, 1227, 1208, 1171, 1160]","[1168, 1134, 1099, 1073, 1056, 1049, 993, 977]","[1537, 1526, 1491, 1436, 1416, 1412, 1387, 1370]"
Management occupations,"[42.472360317578094, 42.524109651454175, 41.42...","[349, 400, 391, 347, 327, 353, 392, 409]","[1429, 1392, 1370, 1351, 1295, 1285, 1248, 1237]","[1236, 1173, 1148, 1139, 1127, 1103, 1036, 1018]","[1585, 1573, 1539, 1486, 1454, 1456, 1428, 1427]"
Chief executives,"[27.959927140255008, 27.55281690140845, 27.652...","[752, 495, 543, 415, 674, 455, 545, 658]","[2291, 2296, 2303, 2041, 2023, 2069, 2060, 1963]","[1736, 1920, 1876, 1836, 1572, 1811, 1730, 1464]","[2488, 2415, 2419, 2251, 2246, 2266, 2275, 2122]"


In [24]:
#df_combined_2010_clean.to_excel('Data/weeklyincome_occupation_gender_combined_clean_2010codes.xlsx')

Combine `df_combined_2002` and `df_combined_2010` by referencing the 2010-2002 occupation code crosswalk. Go through the occupations in `df_combined_2002` and get corresponding 2010 titles and replace (don't delete the ones that don't have a 2010 title because the combining process would remove them if necessary). 

Read in crosswalk and clean occupation names to allow for consistent matching

In [25]:
df_crosswalk = pd.read_excel('Data/2010-2002-occ-codes-crosswalk.xlsx')
df_crosswalk.head()

,2002 SOC code,2002 Census code,2000 SOC title,2010 SOC code,2010 Census Code,2010 SOC title,Unnamed: 6
0,11-1011,10.0,Chief Executives,11-1011,0010,Chief Executives,NaN
1,11-1021,20.0,General and Operations Managers,11-1021,0020,General and Operations Managers,NaN
2,11-1031,30.0,Legislators,11-1031,0030,Legislators,NaN
3,11-2011,40.0,Advertising and Promotions Managers,11-2011,0040,Advertising and Promotions Managers,NaN
4,11-2020,50.0,Marketing and Sales Managers,11-2020,0050,Marketing and Sales Managers,NaN


In [26]:
df_crosswalk['2000 SOC title'] = df_crosswalk['2000 SOC title'].apply(clean_name)
df_crosswalk['2010 SOC title'] = df_crosswalk['2010 SOC title'].apply(clean_name)
df_crosswalk.head()

,2002 SOC code,2002 Census code,2000 SOC title,2010 SOC code,2010 Census Code,2010 SOC title,Unnamed: 6
0,11-1011,10.0,Chief executives,11-1011,0010,Chief executives,NaN
1,11-1021,20.0,General and operations managers,11-1021,0020,General and operations managers,NaN
2,11-1031,30.0,Legislators,11-1031,0030,Legislators,NaN
3,11-2011,40.0,Advertising and promotions managers,11-2011,0040,Advertising and promotions managers,NaN
4,11-2020,50.0,Marketing and sales managers,11-2020,0050,Marketing and sales managers,NaN


In [27]:
df_combined_2002 = pd.read_excel('Data/weeklyincome_occupation_gender_combined_2002codes.xlsx')
df_combined_2002 = df_combined_2002.set_index('Occupation')
df_combined_2002.head()

,2010 Total Number of workers,2010 Total Median weekly earnings,2010 Men Number of workers,2010 Men Median weekly earnings,2010 Women Number of workers,2010 Women Median weekly earnings,2009 Total Number of workers,2009 Total Median weekly earnings,2009 Men Number of workers,2009 Men Median weekly earnings,...,2004 Men Number of workers,2004 Men Median weekly earnings,2004 Women Number of workers,2004 Women Median weekly earnings,2003 Total Number of workers,2003 Total Median weekly earnings,2003 Men Number of workers,2003 Men Median weekly earnings,2003 Women Number of workers,2003 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,99531,747,55059,824,44472,669,99820,739,55108,819,...,57001,713,44223,573,100302,620,56227,695,44076,552
Management professional and related occupations,39145,1063,19009,1256,20136,923,39080,1044,18928,1248,...,17981,1098,18168,780,35680,887,17718,1059,17962,758
Management business and financial operations occupations,15648,1155,8552,1363,7096,971,15879,1138,8635,1334,...,8170,1158,6609,812,14493,961,8047,1143,6446,799
Management occupations,10743,1230,6376,1414,4368,1018,10907,1208,6508,1384,...,6226,1215,3995,871,10115,1023,6143,1172,3973,849
Chief executives,1034,1949,769,2217,265,1598,1105,1916,837,2084,...,802,1875,248,1310,1040,1558,796,1736,244,1243


Change matched occupation titles from 2000 SOC titles to 2010 SOC titles. 

In [28]:
temp_list = df_combined_2002.index.tolist()
for i, occ in enumerate(temp_list):
    occ = clean_name(occ)
    ind = df_crosswalk.index[df_crosswalk['2000 SOC title'].str.contains(occ)].tolist()
    if not ind:
        continue
    else:
        # replace with the 2010 title
        new_occ = df_crosswalk.loc[ind]['2010 SOC title'] # series
        new_occ = new_occ.tolist()[0]
        if new_occ is not np.nan and new_occ != 'Nan':
            as_list = df_combined_2002.index.tolist()
            as_list[i] = new_occ
            df_combined_2002.index = as_list

In [29]:
df_combined_2002.head()

,2010 Total Number of workers,2010 Total Median weekly earnings,2010 Men Number of workers,2010 Men Median weekly earnings,2010 Women Number of workers,2010 Women Median weekly earnings,2009 Total Number of workers,2009 Total Median weekly earnings,2009 Men Number of workers,2009 Men Median weekly earnings,...,2004 Men Number of workers,2004 Men Median weekly earnings,2004 Women Number of workers,2004 Women Median weekly earnings,2003 Total Number of workers,2003 Total Median weekly earnings,2003 Men Number of workers,2003 Men Median weekly earnings,2003 Women Number of workers,2003 Women Median weekly earnings
Total fulltime wage and salary workers,99531,747,55059,824,44472,669,99820,739,55108,819,...,57001,713,44223,573,100302,620,56227,695,44076,552
Management professional and related occupations,39145,1063,19009,1256,20136,923,39080,1044,18928,1248,...,17981,1098,18168,780,35680,887,17718,1059,17962,758
Management business and financial operations occupations,15648,1155,8552,1363,7096,971,15879,1138,8635,1334,...,8170,1158,6609,812,14493,961,8047,1143,6446,799
Management occupations,10743,1230,6376,1414,4368,1018,10907,1208,6508,1384,...,6226,1215,3995,871,10115,1023,6143,1172,3973,849
Chief executives,1034,1949,769,2217,265,1598,1105,1916,837,2084,...,802,1875,248,1310,1040,1558,796,1736,244,1243


Read in values for `df_combined_2010`

In [30]:
df_combined_2010 = pd.read_excel('Data/weeklyincome_occupation_gender_combined_2010codes.xlsx')
df_combined_2010 = df_combined_2010.set_index('Occupation')
df_combined_2010.head()

,2018 Total Number of workers,2018 Total Median weekly earnings,2018 Men Number of workers,2018 Men Median weekly earnings,2018 Women Number of workers,2018 Women Median weekly earnings,2017 Total Number of workers,2017 Total Median weekly earnings,2017 Men Number of workers,2017 Men Median weekly earnings,...,2012 Men Number of workers,2012 Men Median weekly earnings,2012 Women Number of workers,2012 Women Median weekly earnings,2011 Total Number of workers,2011 Total Median weekly earnings,2011 Men Number of workers,2011 Men Median weekly earnings,2011 Women Number of workers,2011 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,115567,886,64142,973,51425,789,113272,860,62980,941,...,57286,$854,45462,$691,100457,$756,55971,$832,44486,$684
Management professional and related occupations,48808,1246,23685,1468,25123,1078,47207,1224,22815,1442,...,19926,"1,328",21059,951,39791,"1,082",19267,"1,269",20524,941
Management business and financial operations occupations,19863,1355,10668,1537,9195,1168,19414,1327,10415,1526,...,9121,"1,387",7869,993,16061,"1,160",8676,"1,370",7386,977
Management occupations,13477,1429,7754,1585,5724,1236,13169,1392,7568,1573,...,6783,"1,428",4765,"1,036",10891,"1,237",6451,"1,427",4440,"1,018"
Chief executives,1098,2291,790,2488,307,1736,1136,2296,823,2415,...,739,"2,275",265,"1,730",990,"1,963",745,"2,122",245,"1,464"


Combine the two

In [31]:
df_combined_all = df_combined_2010
df_combined_all = pd.concat([df_combined_all, df_combined_2002], axis=1, join_axes=[df_combined_all.index])
df_combined_all.head()

,2018 Total Number of workers,2018 Total Median weekly earnings,2018 Men Number of workers,2018 Men Median weekly earnings,2018 Women Number of workers,2018 Women Median weekly earnings,2017 Total Number of workers,2017 Total Median weekly earnings,2017 Men Number of workers,2017 Men Median weekly earnings,...,2004 Men Number of workers,2004 Men Median weekly earnings,2004 Women Number of workers,2004 Women Median weekly earnings,2003 Total Number of workers,2003 Total Median weekly earnings,2003 Men Number of workers,2003 Men Median weekly earnings,2003 Women Number of workers,2003 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,115567,886,64142,973,51425,789,113272,860,62980,941,...,57001.0,713.0,44223.0,573.0,100302.0,620.0,56227.0,695.0,44076.0,552.0
Management professional and related occupations,48808,1246,23685,1468,25123,1078,47207,1224,22815,1442,...,17981.0,1098.0,18168.0,780.0,35680.0,887.0,17718.0,1059.0,17962.0,758.0
Management business and financial operations occupations,19863,1355,10668,1537,9195,1168,19414,1327,10415,1526,...,8170.0,1158.0,6609.0,812.0,14493.0,961.0,8047.0,1143.0,6446.0,799.0
Management occupations,13477,1429,7754,1585,5724,1236,13169,1392,7568,1573,...,6226.0,1215.0,3995.0,871.0,10115.0,1023.0,6143.0,1172.0,3973.0,849.0
Chief executives,1098,2291,790,2488,307,1736,1136,2296,823,2415,...,802.0,1875.0,248.0,1310.0,1040.0,1558.0,796.0,1736.0,244.0,1243.0


In [32]:
#df_combined_all.to_excel('Data/weeklyincome_occupation_gender_combined_all.xlsx')

Clean `df_combined_all`

In [33]:
df_combined_all = pd.read_excel('Data/weeklyincome_occupation_gender_combined_all.xlsx')
df_combined_all = df_combined_all.set_index('Occupation')
df_combined_all = df_combined_all.dropna()

Make sure all monetary values are integers

In [34]:
for col in df_combined_all.columns:
    df_combined_all[col] = df_combined_all[col].apply(clean_val)

In [35]:
df_combined_all.head()

,2018 Total Number of workers,2018 Total Median weekly earnings,2018 Men Number of workers,2018 Men Median weekly earnings,2018 Women Number of workers,2018 Women Median weekly earnings,2017 Total Number of workers,2017 Total Median weekly earnings,2017 Men Number of workers,2017 Men Median weekly earnings,...,2004 Men Number of workers,2004 Men Median weekly earnings,2004 Women Number of workers,2004 Women Median weekly earnings,2003 Total Number of workers,2003 Total Median weekly earnings,2003 Men Number of workers,2003 Men Median weekly earnings,2003 Women Number of workers,2003 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,115567,886,64142,973,51425,789,113272,860,62980,941,...,57001.0,713.0,44223.0,573.0,100302.0,620.0,56227.0,695.0,44076.0,552.0
Management professional and related occupations,48808,1246,23685,1468,25123,1078,47207,1224,22815,1442,...,17981.0,1098.0,18168.0,780.0,35680.0,887.0,17718.0,1059.0,17962.0,758.0
Management business and financial operations occupations,19863,1355,10668,1537,9195,1168,19414,1327,10415,1526,...,8170.0,1158.0,6609.0,812.0,14493.0,961.0,8047.0,1143.0,6446.0,799.0
Management occupations,13477,1429,7754,1585,5724,1236,13169,1392,7568,1573,...,6226.0,1215.0,3995.0,871.0,10115.0,1023.0,6143.0,1172.0,3973.0,849.0
Chief executives,1098,2291,790,2488,307,1736,1136,2296,823,2415,...,802.0,1875.0,248.0,1310.0,1040.0,1558.0,796.0,1736.0,244.0,1243.0


Adjust pay values for inflation using CPI-U-RS taken from Census Bureau. This can be done by converting monetary values to constant 2018 dollars.

First read in CPI-U-RS values from excel

In [36]:
df_cpi = pd.read_excel('Data/CPI_U_RS.xlsx')
df_cpi = df_cpi.set_index('YEAR')
df_cpi.head()

,JAN,FEB,MAR,APR,MAY,JUNE,JULY,AUG,SEP,OCT,NOV,DEC,AVG
YEAR,,,,,,,,,,,,,
1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN
1978,100.5,101.1,101.8,102.7,103.6,104.5,105.0,105.5,106.1,106.7,107.3,107.8,104.4
1979,108.7,109.7,110.7,111.8,113.0,114.1,115.1,116.0,117.1,117.9,118.5,119.5,114.3
1980,120.8,122.4,123.8,124.7,125.7,126.7,127.5,128.6,129.9,130.7,131.5,132.4,127.1
1981,133.6,135.2,136.3,137.1,137.9,138.7,139.7,140.7,141.8,142.4,142.9,143.4,139.1


For each year, multiply the Total Median weekly earnings, Women Median weekly earnings, and Men Median weekly earnings by the corresponding cpi ratio (`cpi_2018/cpi_yr`). 

In [37]:
cpi_2018 = df_cpi.loc[2018]['AVG']
years = range(2018, 2002, -1)
for yr in years:
    cpi_yr = df_cpi.loc[yr]['AVG']
    cpi_ratio = cpi_2018/cpi_yr
    df_combined_all[str(yr)+' Women Median weekly earnings'] = df_combined_all[str(yr)+' Women Median weekly earnings']*cpi_ratio
    df_combined_all[str(yr)+' Men Median weekly earnings'] = df_combined_all[str(yr)+' Men Median weekly earnings']*cpi_ratio
    df_combined_all[str(yr)+' Total Median weekly earnings'] = df_combined_all[str(yr)+' Total Median weekly earnings']*cpi_ratio
    
df_combined_all.head()

,2018 Total Number of workers,2018 Total Median weekly earnings,2018 Men Number of workers,2018 Men Median weekly earnings,2018 Women Number of workers,2018 Women Median weekly earnings,2017 Total Number of workers,2017 Total Median weekly earnings,2017 Men Number of workers,2017 Men Median weekly earnings,...,2004 Men Number of workers,2004 Men Median weekly earnings,2004 Women Number of workers,2004 Women Median weekly earnings,2003 Total Number of workers,2003 Total Median weekly earnings,2003 Men Number of workers,2003 Men Median weekly earnings,2003 Women Number of workers,2003 Women Median weekly earnings
Occupation,,,,,,,,,,,,,,,,,,,,,
Total fulltime wage and salary workers,115567,886.0,64142,973.0,51425,789.0,113272,880.963989,62980,963.938504,...,57001.0,950.152793,44223.0,763.587027,100302.0,848.541821,56227.0,951.188009,44076.0,755.475944
Management professional and related occupations,48808,1246.0,23685,1468.0,25123,1078.0,47207,1253.837119,22815,1477.151247,...,17981.0,1463.208649,18168.0,1039.437838,35680.0,1213.962250,17718.0,1449.364175,17962.0,1037.410807
Management business and financial operations occupations,19863,1355.0,10668,1537.0,9195,1168.0,19414,1359.347922,10415,1563.198892,...,8170.0,1543.165405,6609.0,1082.081441,14493.0,1315.239822,8047.0,1564.327905,6446.0,1093.524056
Management occupations,13477,1429.0,7754,1585.0,5724,1236.0,13169,1425.932410,7568,1611.344598,...,6226.0,1619.124324,3995.0,1160.705586,10115.0,1400.094004,6143.0,1604.017765,3973.0,1161.954848
Chief executives,1098,2291.0,790,2488.0,307,1736.0,1136,2351.968975,823,2473.869806,...,802.0,2498.648649,248.0,1745.722523,1040.0,2132.303479,796.0,2375.917098,244.0,1701.189489


In [38]:
#df_combined_all.to_excel('Data/weeklyincome_occupation_gender_combined_all_adjusted.xlsx')

Consolidate years into arrays

In [39]:
df_combined_all_clean = consolidate_df(df_combined_all, range(2018, 2002, -1))
df_combined_all_clean.head()

,Percent Female,Pay Gap,Total Weekly Pay,Female Weekly Pay,Male Weekly Pay
Occupation,,,,,
Total fulltime wage and salary workers,"[44.497996833005956, 44.39843915530758, 44.252...","[184.0, 175.16842105263163, 173.70345217883425...","[886.0, 880.9639889196677, 870.6100735710244, ...","[789.0, 788.7700831024931, 783.7583474816072, ...","[973.0, 963.9385041551247, 957.4617996604414, ..."
Management professional and related occupations,"[51.47311916079331, 51.672421462918635, 51.543...","[390.0, 399.50692520775624, 411.23769100169784...","[1246.0, 1253.8371191135736, 1243.130730050933...","[1078.0, 1077.64432132964, 1074.6593095642331,...","[1468.0, 1477.1512465373962, 1485.897000565931..."
Management business and financial operations occupations,"[46.29210089110406, 46.35314721335119, 46.0547...","[369.0, 401.5556786703603, 410.19128466327106,...","[1355.0, 1359.3479224376733, 1343.585738539898...","[1168.0, 1161.643213296399, 1150.0005659309563...","[1537.0, 1563.1988919667592, 1560.191850594227..."
Management occupations,"[42.472360317578094, 42.524109651454175, 41.42...","[349.0, 409.75069252077583, 409.1448783248445,...","[1429.0, 1425.9324099722994, 1433.576683644595...","[1236.0, 1201.5939058171746, 1201.274476513865...","[1585.0, 1611.3445983379504, 1610.419354838709..."
Chief executives,"[27.959927140255008, 27.55281690140845, 27.652...","[752.0, 507.06648199446, 568.1986417657045, 43...","[2291.0, 2351.9689750692523, 2409.873797396717...","[1736.0, 1966.8033240997233, 1963.058290888511...","[2488.0, 2473.8698060941833, 2531.256932654216..."


In [40]:
#df_combined_all_clean.to_excel('Data/weeklyincome_occupation_gender_combined_clean_all.xlsx')